In [2]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [4]:

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

In [32]:
connection.rollback()

query = """

drop table if exists stations;
drop table if exists lines;
drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


In [33]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [34]:
connection.rollback()

query = """

copy stations
from '/user/projects/project-3-AppleTater/exercise/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [35]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

In [36]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-AppleTater/exercise/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [37]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

In [38]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-AppleTater/exercise/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


In [13]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
where station = 'West Oakland'
order by station, line

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,West Oakland,blue
1,West Oakland,green
2,West Oakland,red
3,West Oakland,yellow


In [14]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
order by station, line

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
where a.station = 'West Oakland'
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,West Oakland,blue,green,283
1,West Oakland,blue,red,283
2,West Oakland,blue,yellow,283
3,West Oakland,green,blue,283
4,West Oakland,green,red,283
5,West Oakland,green,yellow,283
6,West Oakland,red,blue,283
7,West Oakland,red,green,283
8,West Oakland,red,yellow,283
9,West Oakland,yellow,blue,283


In [16]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where (a.station = 'West Oakland' or b.station = 'West Oakland')
order by line, from_station, to_station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,blue,Lake Merritt,West Oakland,360
1,blue,West Oakland,Embarcadero,420
2,green,Lake Merritt,West Oakland,360
3,green,West Oakland,Embarcadero,420
4,red,12th Street,West Oakland,300
5,red,West Oakland,Embarcadero,420
6,yellow,12th Street,West Oakland,300
7,yellow,West Oakland,Embarcadero,420


In [18]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300


## Finish creating table

In [119]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [6]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [7]:
session = driver.session(database="neo4j")

In [45]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [9]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    pd.set_option('display.max_rows', 500)
    
    return df

In [10]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [11]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [12]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [13]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [14]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [15]:
cursor = connection.cursor()

In [ ]:
### create data base

In [120]:
my_neo4j_wipe_out_database()

In [121]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [122]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [123]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [124]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [125]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [126]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [127]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [128]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


In [ ]:
################ finish up ##########################

In [83]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [81]:
query = """

MATCH (n) where n.name =~ '.*arrive.*' DETACH DELETE n

"""
session.run(query)

In [82]:
query = """

MATCH (n) where n.name =~ '.*depart.*'  DETACH DELETE n

"""
session.run(query)

In [102]:
query = """

CALL gds.louvain.stream('ds_graph',{maxLevels: 5})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

df2 = my_neo4j_run_query_pandas(query)


In [106]:
display(df2)
##Result for 10 different cummunity, save as pandas DF for later source

,name,community,intermediate_community
0,yellow Antioch,52,None
1,yellow Concord,52,None
2,yellow Lafayette,52,None
3,yellow North Concord,52,None
4,yellow Orinda,52,None
5,yellow Pittsburg,52,None
6,yellow Pittsburg Center,52,None
7,yellow Pleasant Hill,52,None
8,yellow Rockridge,52,None
9,yellow Walnut Creek,52,None


In [109]:
df2[['code','name_of_code']] = df2["name"].str.split(" ", n=1, expand=True)

In [112]:
df3 = df2[['name_of_code','community']].drop_duplicates()

In [116]:
df3.shape
# Ensure final df only contains required 50 stastion.

(50, 2)

In [117]:
display(df3)

,name_of_code,community
0,Antioch,52
1,Concord,52
2,Lafayette,52
3,North Concord,52
4,Orinda,52
5,Pittsburg,52
6,Pittsburg Center,52
7,Pleasant Hill,52
8,Rockridge,52
9,Walnut Creek,52


In [129]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [138]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        total_population += population
        
    return total_population

In [131]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [132]:
cursor = connection.cursor()

In [134]:
import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

In [139]:
my_station_get_zips('Downtown Berkeley', 1)

Decimal('82930')

In [159]:
df3['population'] = df3.apply(lambda x: my_station_get_zips(x["name_of_code"],2), axis=1)

In [160]:
display(df3)

,name_of_code,community,population
0,Antioch,52,66933
1,Concord,52,87117
2,Lafayette,52,29639
3,North Concord,52,58689
4,Orinda,52,19341
5,Pittsburg,52,0
6,Pittsburg Center,52,96081
7,Pleasant Hill,52,86258
8,Rockridge,52,151725
9,Walnut Creek,52,43556


In [161]:
df3[['community','population']].groupby("community").sum().sort_values(by=['population'])

#View surrounding population by community

,population
community,
75,457639
92,474099
110,485126
67,502005
52,639339
93,665486
74,700050
54,772056
106,825595


In [ ]:
# 74,54,106,98 cummunity chosen base on population

# Detail as below:
# 74:
# 33	Ashby	74	173897
# 34	Downtown Berkeley	74	153753
# 35	El Cerrito Plaza	74	70450
# 36	El Cerrito del Norte	74	81472
# 37	North Berkeley	74	134802
# 38	Richmond	74	85676

# 54:
# 10	Balboa Park	54	253123
# 11	Daly City	54	231056
# 12	Glen Park	54	287877

# 106:
# 89	Embarcadero	106	210921
# 90	Montgomery Street	106	268802
# 91	Powell Street	106	303556
# 92	West Oakland	106	42316

# 98:
# 77	16th Street Mission	98	339093
# 78	24th Street Mission	98	315201
# 79	Civic Center	98	347795

